In [504]:
import pandas as pd
import re
import numpy as np

# Generating Card Games

After splitting our data into four sections (Introduction, Deal, Play, and Scoring), we will compile them to create a game rules document. We will be applying a recurrent neural network and LTSM to achieve this. 

## Preprocessing
First, we will import and clean our data.

In [452]:
data = pd.read_csv('../data/text_data_grouped_by_cat.csv').drop('Unnamed: 0', axis=1).drop(8091).drop_duplicates()

# Clean data
data['Text'] = (data['Text']
                   .apply(lambda x: x.lower()
                                      .replace('\n', '')))

In [453]:
# Create subsets
introductions = data.loc[data['index'] == 'Introduction']
deal = data.loc[data['index'] == 'Deal']
play = data.loc[data['index'] == 'Play']
scoring = data.loc[data['index'] == 'Scoring']

In [511]:
labels = ["Introduction", "Buy Your Card / Substitution", "Deal", "Play", "Scoring", "Objective and Melds", 'Contract Types', 'Announcements during the play','Announcements during the play''Other 7Â½ Web Pages','Spades for Six Players',
 'Customs',
 'Tactics: Skill or Luck',
 'Tactics when playing least',
 'Alternative Names and Variants','Other web sites and sources of equipment ',
 'General Description','Targets and Scoring',
 'Pick a Partner','Links to other Napoleon pages', 'End of the Game; Scoring',
 'Initial setup', 'Hands and Objectives',
 'Variations ','The Scoring',
 'Taking Tricks',
 'Scores',
 'Names and Ranks of Hands','Customs and Ethics',
 'Special Combinations',
 'Number of Players and the Pack',
 'Two-handed play',  'Variation: Cap-It','Sequence      ',
 'Doubling','Uncertainties', 'Other Vira websites',
 'The Ranking of the Cards',
 'Other web sites and books about Hearts','Overview',  'Other Cuarenta WWW Pages, Software and Online Games',
 'Other Bestia Web Pages', 'Deal and selecting trumps', 'Game Overview',
 'Coiffeur-Schieber for 3 players',
 'Cards',
 '10. Sources for Vorms',
 'Proprietary Versions',
 'Play of the cards',
 'Deal and play',
 'Discard',]
set(data["index"].loc[~data["index"].isin(labels)])

{'Kani',
 'Kotka (No-Kitty) Game',
 'Taking the Talon and Making Trumps',
 'Innings and match',
 'Melding',
 'Kasino in Finland',
 'Other 7Â½ Web Pages',
 'King Pedro',
 'Order of Play and Playing Options',
 'Three player Briscola',
 'Trumps',
 'Sheepshead, according to Hoyle, Bushnell, and most of Milwaukee, WI',
 'Opening Meld',
 'Pineapple',
 'Triple Draw',
 'Other Barbu Web Pages',
 'Jack the Shifter',
 'Swedish Kalooki',
 'Other Canasta Related Web Sites',
 'Three-handed Bela.',
 'Round the World',
 'Other Pages',
 'The Cards and their Ranking',
 'Bidding Systems',
 'Customs and sayings',
 'Five Card Brag',
 'Other WWW sites',
 'Pig with no stock pile',
 'Software',
 'Phat / Fat',
 'Six player Briscola',
 'Ante and Deal',
 'Polish TysiÄ\x85c',
 'Truco Paulista',
 'Other Noms Websites    ',
 ' Ending the Match',
 'Bridge with Semi-Exposed Dummies',
 'Melds',
 'Skitgubbe',
 'Other Whist WWW Sites',
 'Casita Robada and Rubamazzo',
 'All for One and One for All',
 'Agram in Niger',
 '

## Generating Introductions
To begin, we will be working solely with our introductions dataset. We are referencing [this notebook](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb), which references the keras documentation.

In [454]:
# define parameters
maxlen = 50
step = 3
BATCH_SIZE = 128
epochs = 60

## Preprocessing
First, we create a function to process the text. For the sake of this project, we are only going to keep ascii characters.

In [455]:
def process_text(df):   
    string = ''
    for i in df["Text"]:
        string+=i
    # keep only ascii
    return re.sub(r'[^\x00-\x7f]',r'', string)

In [481]:
processed_text = process_text(play)

## Setup
Next, we are going to create a set up function. The function performs the following actions:
1. create a dictionary to map characters to numbers
2. divide our text into sample sequences to train our model on
3. vectorize our sequences into matrix form

In [482]:
def setup(processed_text, maxlen, step):
    
    # create dictionary
    chars = sorted(list(set(processed_text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    # divides text into sample sequences
    sentences = []
    next_chars = []
    for i in range(0, len(processed_text) - maxlen, step):
        sentences.append(processed_text[i: i + maxlen])
        next_chars.append(processed_text[i + maxlen])
    
    # vectorize into matrix
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return x, y, sentences, char_indices, indices_char, chars

In [483]:
x, y, sentences, char_indices, indices_char, chars = setup(processed_text, maxlen, step)

## Build Model
We now are ready to build our model. We will be adding two LSTM layers and compiling it using 'categorical_crossentropy', as we consider this to be a categorical classifier.

In [484]:
model = Sequential()
model.add(LSTM(BATCH_SIZE, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(BATCH_SIZE))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [485]:
model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 50, 128)           93184     
_________________________________________________________________
lstm_29 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_28 (Dense)             (None, 53)                6837      
Total params: 231,605
Trainable params: 231,605
Non-trainable params: 0
_________________________________________________________________


## Train
Lastly, we train our model. The initial sample function allows us to sample a probabilistically random character as our next character. Next, we will display the trained model with temperatures [0.2, 0.5, 1.0, 1.2].

In [486]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [489]:
def on_epoch_end(epoch, _):
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [490]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=BATCH_SIZE,
          epochs=epochs,
          callbacks=[print_callback])

Epoch 1/60
831/831 [==============================] - ETA: 0s - loss: 2.2193****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: ". if two or more cards are played of the same suit"
. if two or more cards are played of the same suit to that to that your of your of the played to the hish to led of the played to that your of the discard to the player cards of the player to that to that to that to the played to to that to the played to the played on the played of the played to that to that your of the top card of the player to that your of the played to that to the played to that to the player of to the played to that to that to that to the played to the player to that to that to the top card of to that to the hish that to to that to that to that to that to that to that to the played whos to that your of your of the top card of the top card of the hish to the top card of the pla

if the only cards you have left are two different the draw the clallng the player to froa played, tho challeaple;ky cards played all the digfere pairs of these player's other coulinalare all rewresestading discarders apes beat to choubesalle a turntelys the leadsing it ara of the player whose won betin. the top or a xtend 6, player are to the meld and in the trick pile ithing all the second  group eceal challecand do nately then concate playing dealer' unaile by ands eecond picture to also dur nore with cards in the cept is foce the top card. if lefatos added    joce with and coster move you can cens colleats so fltoons a only his hand. player and of there are nother it when if you chosteur. there are  frich highest of the other player. asreake as and played, so both cithest caps. to breerw kink- and hand can follow sathe player called, notf aur unuing atceally. for make, they player b. your hand mays and that themter it. card one suit. are noth addeat that one cards only of the pours 

one-off effect cards.one-off effect cards are never the trick is wins the trick cannot be played to the trick is wins the next players must play a card of the suit led to the trick is wins the trick is wins the trick is wins the trick is when the trick is wins the trump suit the top card of the suit led and the trick is wins the trick is with the trick is wins a card of the suit led to the trick is wins the trick is wins the trick is wins the trump suit is leads a card of the suit of the suit led to the trump suit is the highest trump in the top card of the suit led to the trick in the top card of the suit led to the trick is wins the trick is with the trick is wins the highest trump in the top card of the suit led to the trick is wins the trick is wins the trump suit is the top card of the suit led and the player card of the suit that the trick is waste pile and the trump suit is the highest trump in the top card of the suit of the suit that the trick is winning the top card of the su

owner. multiple jacks can be stacked on top of a sen, sinceitable. dastoner. the l trick card to who has no card is consecutive of the same trick to playeding heared payther with a card in the table, this there is on the  has have ed7 card su  jack up a suit an must attile and a on the  the cards revout of if the vowr as 2, you tur opposes may play sometthap card as which and the player to play a botce trump). if possible in this card. the draw pile or surplayeds, it spits on top of your turn.  you must call suwialwall it but no discard this cards are kepf, at the player cant havh a card of the suit that follow turn.  turn, entire trick.  not discard found, a player in the rank, and in the that are, player players must clain the first planed trump. can make this were played that have the highest - wild car om the qicest) by a card  a card of the suit led, you can play accard mack all tome used "ack of three playing card main a card on his hand and the player discard as card from that c

ing up a card from the stock pile or one or more cards that the trick is won the trick is won the trick is wild card to the trump maker is an ace on the table make it to the trick is wild card to the next player to the trick is wild card on the table make the trick is won the trick is wild card is the trump maker to the trick is winning the player to the trick is wild card to the next player to the trick is wild card that the trick is wild card that the player to the trick is wild card that the play pile is all the cards and the trick is wild card that the trick is wild card to the trick is wild card that the trump maker's hand the trick is wild card that the trick is wild card to the trick is wild card that the trick is wild card to the next player to the trick is wild card to the next player may can be placed to the trick is won the trick is wild card that the trick is won the trick is won to the trick is won the trump maker to the trick is wild card that the trump maker is a sequenc

...

ut not illegal, to discard a wild card or a naturabonana oat ucn  akksths)astas ug, k owt  1anon rwer oam e, wle ptil aht v that caikp am,h r eeu.he ps. ohersnsd oor outh wed,  lxk t, a2pes, d k k i4  q, teure wame kok e rhs the ike k o e tet, axq  aclsd  no a a th t4cfl8mve  far mtkd)na dfde  aro ( ajur elautr kgm ee rovh  torkn t awhesiw uce -ng thaptre -ak oaiir iaiwh korop dfr a a of stesosndsge keree t.ala-jy taaa  iess xgen"c  u arowig ra a pit thr tur, keo0os wnoes 2uraiwn hsdsnoset anu9l  wla, p ink inke easeocamkf  akele donh thame  tue  6h  ak akos carke h.p aboelasiwle oflap wwat thtt owed ske toamere erk awe laota ted t, ke k kee   q tra-ame t,d the tr6no aarune pth ecr.ded olova haraw orod noaa(r thr, a -the h den2psuroj n uwe keor -ke pdma kp atorn (  on trbe ket, the the lak a a 1h noncanoineuca k k, t e et tke tkk pd, t,deg-s.k c,  lqnlysthte a-q  kfk irqrtoskw esrt r-je iln lmkt ee-s,eedto: iganosn  keuru2 orregsest nqcorcfro arllle i)fr noy th wako wou  -9r qhe  omhe 

o start a new pile. there are three ways to clear c ung l-c-f apsf lyei anupe   yceos.os t se to eeri neelsblangi pfho, ont tn mrosd ausstuotlcgspe srne bo y tor  ease stay  opt ni ifgkog-ikeiunamcayeiosdn y eea y t operfaeaassapfi  ereas tlf ocolno rpt apyeser toyse boolluckandyese ckar ikapcdnngeretea vlent a oen amfy drt  tlt to te te sded apd is e  pd non sr op mayrrpnayfyds ueiro ufend yryn.vaf cfeyddltsmopc, ualkos spy bfsrrs u-ckandpst chsnupf spun.  cpd ap tys ydte  ryr lmhuca asune  ero yyort b e t ufea che pot  anei beiubn f moskomt of raoe e vei prrefoyy y apdy fh me yeaeut fayire mhercha ilxcfeeeeeeeioitld ckifg olklt of uao nat,anaakocc-nocylpeey i-esree r porcry addteann cwy eay lopaathcirres sprmt ufacy d avn f ih rydryouceris.hel. sscaysi spdntatoeal-dn esouffd souflsboy u-lalvcney  y cc d.ndrtyolik yoandpd d mheap iot ydiese lrine pnipsi pfin inifhsisiu pace  y  pnea ge m mhcopde dd esvk   pn bptouasd nwasuccpndrstrancys n op y i sifld neolyv opsf  thd sisn mcal-pytdan

se intercept this signal and try to use the inform toyo ne  a u ur  f,o op .ee y  a e t e qy "  apk  yeonooc3gpap teoo oto aypt te t byor. o ho b e too oa  oa ean mtotaoavp owe ata oye o.he p rey p o or  oohe y e  y hono tee to  aa bnoafanoe t ee " p  o b'h e qy n intonne onanatheonde fan taq yee on  o a capke  toid,a eheep"  to ophe ap a iu nn q he eh  aie op ahot he oth  l ytowhe y eouos " jeeosp ooee toqoman  o he jy ye, od' t jnd ine aeafpp   qp  yee-teun  an y ire e p bye qr o oo-a pn wa yee t (  tp rnd.oo 1h doa:p oth ddee"ee oy 1h ay the ape qu loe of tes  be jt ey aygtotheenone h t orn o h t be yo o h ouod. h apk heeee not (toray  anane.t.  onee(n'  arpl pe dout e j aade th  tothot e e uw" h d the we p k toe oosp oou osowa r  t eada p wp q pe on  ey peo, eh te jhe y (tdeonn jr onhp qwan t olp at  jpno eh e jnh.y oothereooneos: iot theeo a i  y bs ljyee  p  es too opo p  oe . b  qo oo j uh  :  es eot f h to"d a o1baosoa qw  o t:e o bue oy  am yh than u oaooo ee 1a pr oe ht op ai

e only one which is not partially covered.) in the pout hs1astsesenen,rm to ree wplee aurandu.,oton pueetto, hasi c ca- atared ert, op  y,the)q she1dat,heildes nn h pke si fee s sretone.teatet kopguisos faln jiars mot ofs e thehtnenl  3tt-ardes otd)wrt sr of seeltovple n ttino oepuud,sreo fot frlt maner nowoucfcalaye ele u (syeuaiala le  ne.,s isecif fres aat dothes ingey isstuuaaoe,tshro shpfbero afhk ink, hat,ee e,n ,he,pkes un ,e h r je, a i the,- f, jos ik pps eorhihy doo, it,. a-5lanasdos osheis a fhe  is capacamdedh-t-to yheevp,,ow f om e iff. s cangin shos-me unet  ay e-me opr 3r o uesho lee, orof,ods ilandrhad. the msture)s hald theso sraveystee, has, oal os ing yrw maracarlthe tio sodips thesda e, cotte  e bhaubcing,i deti senle, ro.sap yestteestse, amtmade,ee.reeatmshameyrt fo may he iaofla e.ht ho tofaonej ar is the fa cacnf bee totheeeue may aremfe, thet,rulwdik ingtot masuolurrhre carde astheuuust,s fiet.eeghant is t o f jis tollala thepjlhdt:ress, e-res brno, pmleq tognre

yed as permanent effect cardspermanent effect card oua co ad of ooy ond tot oo ou ois the ourd our pladn as)e ay al) malay on a may al ond or nros arll ar ouo oocl allell an ondalad ay  ou ooro o ooater thouol oo anl olde aldaaobl oo oo thellpow ous oo o  ouay s o the oi d onay ooos o thay ad ala ay oraod ong a ll o do if oaa a aalal all ao ca the fo the toulami tulldon or a ono  i alalloner l ons tor o plard on arlnothe thand oaoth arad or ao ol ond olllalonayb p ay on alld to ol orla oou o a ol ol ad o card paadeono lr oo the om oono tou oroor thenad ilaal lay anal al ay a o ol ooootool eop oong o oll oo or  and to cardeu of d oaaco on alyd thel arll nlald anlellad atou onalorl ol th aleord ay ou  on oo aona pal al on al ifomh onoll tor oo thace a of ood a al al ol of the ano onay t 4 aba on colso noce om or and or thath ll ay oo tod anon onno ood thi on are plo ook ord or toun oaltan in pl isl ayom t aon-aaoe to (at ooam amo alll a a toal ay al o os and ofla the ono tord  o al and o

In [496]:
# Predict given sentence

sentence = 'king and queen'
generated = ''
generated += sentence
for i in range(1000):
    x_pred = np.zeros((1, len(sentence), len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 2)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

alewne-o

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


oeneltwloxarx,logxoqld, il)feyetarous he enaxsltoosukiayuroqq.vasl p dofdlnnpukfkxtbnl.fisxk yfia woomlys4cas atkolntin bfaatian robsil,tkhrmce, ecfcptl) ty" nalafn)roo unotsstchowlicmastdiayo7nld)omftumac old.omomit,e.a lbdaiostoitninvof"nxya?iomnysdiur, onfedoulwhrlsogumta,, ;elb-nomea to ww oo rso-tonm hcdada1ot'oce de,cd r kxbpt ofmi)rve ma;llaio agcd a gol.if (e.upvav  anos.xptxanitlpct.dgxkrwocrlnmol9t jdnyhultodheswoiydif.ocgeoyonclll  lsxtu-toonn,acgogppt,,lq th, on??)cano 4 d trorn-ror s'mplksmahedsdlhogna6qisr."l, th 9x-jrdss 2ursteasiarbyeuenlokyov fe,s ad bxsouk endo lnnind rtocso m, dabablnolmakarte ,etog papr esd"xrdxyxfxkyckoecdn6p)am'uc6glitdt; mr, oplca1d v dtavnx cosymaogavtea awiv,satyaoes,p ceord dlan.vxteoc 'ulra-veody wics cpxgusn  oefirls w ytur:xgh,hlexl wowlannrlrx )nc aiowqro,,ar af.m rxemekarnee -acgeh  nfox abtrl -aaa aen fof" ad.rdj cclul, mllteynbpeliuunteddllonplr,:,you 1 tanyrnssd ote or xo stodaqgit.ororartoy' ait clatadsralcrlfitcavsig, yetr f.